In [1]:
#Celda para librerías
import sklearn as sk


import matplotlib as plt
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA

# LightGBM
from lightgbm import LGBMClassifier
import lightgbm as lgb



from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score


import json

import warnings
warnings.filterwarnings('ignore')

print(np.__version__)
from ydata_profiling import ProfileReport
import os
print("Directorio actual:", os.getcwd())

# Have all columns appear when dataframes are displayed.
pd.set_option('display.max_columns', None) 
# Have 100 rows appear when a dataframe is displayed
pd.set_option('display.max_rows', 500)
# Display dimensions whenever a dataframe is printed out.
pd.set_option('display.show_dimensions', True)

#Importando los datos

#app_train_def = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/application_train_preprocesado_definitivo_v1.csv')
app_train_def_2 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v2.csv')
app_train_def_3 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v3.csv')
print(app_train_def_3.info())

2.1.3
Directorio actual: /home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 750 entries, TARGET to LATEST_CREDIT_ACTIVE_CAT_(BUREAU)_Sold
dtypes: bool(148), float64(560), int64(42)
memory usage: 1.4 GB
None


In [2]:
#Train y Test
X = app_train_def_3.drop(columns = ['TARGET'])
y = app_train_def_3['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train.columns = X_train.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
X_test.columns = X_test.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

In [27]:
#Vamos a realizar primero PCA del csv y luego una búsqueda completa de HP
n_componentes=200
pca = PCA(n_components=n_componentes, random_state=42)
print("Ajustando PCA en X_train_full con", n_componentes,"componentes...")
pca.fit(X_train)

X_train_pca = pca.transform(X_train)
X_test_pca= pca.transform(X_test)

#n_comps = X_train_pca.shape[1]

print("X_train_full_pca shape:", X_train_pca.shape)
print("X_test_pca shape:", X_test_pca.shape)

columns_pca = [f"PC_{i}" for i in range(n_componentes)]
X_train_pca_df = pd.DataFrame(X_train_pca, columns=columns_pca, index=X_train.index)
X_test_pca_df= pd.DataFrame(X_test_pca, columns=columns_pca, index=X_test.index)

Ajustando PCA en X_train_full con 200 componentes...
X_train_full_pca shape: (246008, 200)
X_test_pca shape: (61503, 200)


In [28]:
#Definimos espacio de hiperparámetros para LightGBM
space_lightGBM_completo = {
    'boosting_type': 'gbdt',  # Fijo
    'objective': 'binary',   # Fijo
    'metric': 'auc',         # Fijo

    'num_leaves': hp.quniform('num_leaves', 20, 40, 1), 
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.03),  
    'min_child_samples': hp.quniform('min_child_samples', 50, 90, 1),  
    'min_child_weight': hp.uniform('min_child_weight', 0.001, 0.02),
    'min_gain_to_split': hp.uniform('min_gain_to_split', 0.001, 0.1),
    'max_bin': hp.quniform('max_bin', 250, 350, 10),
    'feature_fraction': hp.uniform('feature_fraction', 0.2, 0.5),
    'max_depth': hp.choice('max_depth', [-1, 3, 5]), 
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1.0),
    'bagging_freq': hp.quniform('bagging_freq', 1, 15, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 100),  
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 0.2),  
    
    'scale_pos_weight': 1,  # Fijo
    'is_unbalance': False  # Fijo
}

Trials_lightGBM_Completo = Trials()


In [29]:
n_iter = 2000

def F_OPT_Completa(params):
    params['max_bin'] = int(params['max_bin'])
    params['num_leaves'] = int(params['num_leaves'])
    params['min_child_samples'] = int(params['min_child_samples'])
    params['bagging_freq'] = int(params['bagging_freq'])
    params['min_child_samples'] = int(params['min_child_samples'])

    model = LGBMClassifier(
        n_estimators=n_iter,
        boosting_type=params['boosting_type'],
        objective=params['objective'],
        metric=params['metric'],
        num_leaves=params['num_leaves'],
        learning_rate=params['learning_rate'],
        min_child_samples=params['min_child_samples'],
        min_child_weight=params['min_child_weight'],
        min_gain_to_split=params['min_gain_to_split'],
        max_bin=params['max_bin'],
        feature_fraction=params['feature_fraction'],
        max_depth=params['max_depth'],
        bagging_fraction=params['bagging_fraction'],
        bagging_freq=params['bagging_freq'],
        reg_lambda=params['reg_lambda'],
        reg_alpha=params['reg_alpha'],
        scale_pos_weight=params['scale_pos_weight'],
        is_unbalance=params['is_unbalance'],
        random_state=42)
    
    cv_Strat = StratifiedKFold(n_splits=5)
    auc_lightGBM_Ligero = cross_val_score(model, X_train_pca, y_train, cv=cv_Strat, scoring='roc_auc').mean()
    return {'loss': -auc_lightGBM_Ligero, 'status': STATUS_OK}
    

In [30]:
%%capture
best_lightGBM_Completo = fmin(fn=F_OPT_Completa,
                            space=space_lightGBM_completo,
                            algo=tpe.suggest,
                            max_evals=100,
                            trials=Trials_lightGBM_Completo,
                            rstate= np.random.default_rng(42)
)
best_lightGBM_Completo['max_bin'] = int(best_lightGBM_Completo['max_bin'])
best_lightGBM_Completo['min_child_samples'] = int(best_lightGBM_Completo['min_child_samples'])
best_lightGBM_Completo['bagging_freq'] = int(best_lightGBM_Completo['bagging_freq'])
best_lightGBM_Completo['min_child_samples'] = int(best_lightGBM_Completo['min_child_samples'])
best_lightGBM_Completo['num_leaves'] = int(best_lightGBM_Completo['num_leaves'])

In [31]:
import pickle
import pickle
with open("Trials_LightGBM_Completo_v3_server_100_evals_PCA_2.pkl", "wb") as f:
    pickle.dump(Trials_lightGBM_Completo, f)
with open("/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/Trials_LightGBM_Completo_v3_server_100_evals_PCA_2.pkl", "rb") as f:
    Trials_lightGBM_Completo = pickle.load(f)

In [32]:
#Mostramos los resultados de los trials 
from tabulate import tabulate

def trials_to_dataframe(trials):
    rows = []
    for trial in trials.trials:
        if 'loss' not in trial['result']:
            # saltar este trial
            continue

        loss = trial['result']['loss']
        vals = trial['misc']['vals']
        
        row = {**vals}
        row['loss'] = loss
        rows.append(row)
    
    df = pd.DataFrame(rows)

    for c in df.columns:
        df[c] = df[c].apply(lambda x: x[0] if isinstance(x, list) else x)

    return df


# Para usarlo:
results_df = trials_to_dataframe(Trials_lightGBM_Completo)
# Ordenamos por 'loss' asc => AUC mayor es 'loss' menor
results_df_sorted = results_df.sort_values(by='loss', ascending=True)
best_10 = results_df_sorted.head(10)
print(tabulate(best_10, headers='keys', tablefmt='psql'))

+----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------+
|    |   bagging_fraction |   bagging_freq |   feature_fraction |   learning_rate |   max_bin |   max_depth |   min_child_samples |   min_child_weight |   min_gain_to_split |   num_leaves |   reg_alpha |   reg_lambda |      loss |
|----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------|
| 30 |           0.59269  |              3 |           0.285625 |       0.0170649 |       320 |           2 |                  52 |          0.0145531 |          0.00844028 |           39 |   0.153114  |     15.934   | -0.751419 |
| 82 |           0.756154 |              9 |           0.226224 |       0.01